# Semantic Search

## Import Libraries

In [10]:
$ pip install pinecone-client

SyntaxError: invalid syntax (2138097809.py, line 1)

In [8]:
from datasets import load_dataset
import torch
from dotenv import load_dotenv, find_dotenv
from sentence_transformers import SentenceTransformer
import os
from pinecone import Pinecone

_ = load_dotenv(find_dotenv("secrets.env", raise_error_if_not_found=True))

ROOT_DIR = os.environ["ROOT_DIR"]

ModuleNotFoundError: No module named 'pinecone'

## Load and Transform the dataset

In [61]:
# Load the dataset
with open(file=f"{ROOT_DIR}/Pinecone Building Applications with Vector Databases/files/datasetText.txt") as file:
	dataset = file.read().split("\n")[1:]
# dataset[:20]

# Extract the questions from the dataset
import re
questions = []
for data in dataset:
	result = re.findall(r"'text': \[([^\]]+)", data)
	result = list(result)
	for q in result:
	# 	q = q.split(", ")
		pattern = r'([\'"])(.*?)\1'
		q = re.findall(pattern, q)
		for e in q:
			garbage, question = e
			questions.append(question)

print("\n".join(questions[:5]))
print("-"*50)
print(f"Number of questions = {len(questions)}")

# Not exactly 100000 questions, but that should do it

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
--------------------------------------------------
Number of questions = 100041


## Instantiate the Model

In [3]:
# See if a GPU is available
if torch.cuda.is_available():
    device = "cude"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# Load the model on the GPU
modelName = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name_or_path=modelName, device=device)
print(f"Model: {modelName}\nRunning on: {device}")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Model: all-MiniLM-L6-v2
Running on: mps


## Encoding Queries

In [5]:
query = "what is the most populated city in the world?"
xq = model.encode(sentences=query)
print(f"Shape of xq: {xq.shape}\nType of xq: {type(xq)}")


Shape of xq: (384,)
Type of xq: <class 'numpy.ndarray'>


## Setting up Pinecone

In [6]:
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]